In [2]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import numpy as np
#这个应该是要用的那个CNN，到时候改一下就可以，这个其实就是一个TDNN_layer
#self.padlen = int(dilation * (context_size - 1) / 2)
class TDNN(nn.Module):
    def __init__(self, num_speakers):
        super(TDNN,self).__init__()
        self.TDNN1 = nn.Sequential(
            nn.Conv1d(in_channels = 39, out_channels = 512, kernel_size = 5, padding = 1, dilation = 1),
            nn.ReLU(),
            nn.BatchNorm1d(512, momentum=0.1, affine=True, track_running_stats = True),
            nn.Dropout(p = 0.5)
        )
        self.TDNN2 = nn.Sequential(
            nn.Conv1d(in_channels = 512, out_channels = 512, kernel_size = 3, padding = 2, dilation = 2),
            nn.ReLU(),
            nn.BatchNorm1d(512, momentum=0.1, affine=True, track_running_stats = True),
            nn.Dropout(p = 0.5)
        )
        self.TDNN3 = nn.Sequential(
            nn.Conv1d(in_channels = 512, out_channels = 512, kernel_size = 3, padding = 3, dilation = 3),
            nn.ReLU(),
            nn.BatchNorm1d(512, momentum=0.1, affine=True, track_running_stats = True),
            nn.Dropout(p = 0.5)
        )
        self.TDNN4 = nn.Sequential(
            nn.Conv1d(in_channels = 512, out_channels = 512, kernel_size = 1),
            nn.ReLU(),
            nn.BatchNorm1d(512, momentum=0.1, affine=True, track_running_stats = True),
            nn.Dropout(p = 0.5)
        )
        self.TDNN5 = nn.Sequential(
            nn.Conv1d(in_channels = 512, out_channels = 1536, kernel_size = 3, padding = 3, dilation = 3),
            nn.ReLU(),
            nn.BatchNorm1d(1536, momentum=0.1, affine=True, track_running_stats = True),
            nn.Dropout(p = 0.5)
        )
        self.fc1 = nn.Linear(2*1536,512)
        self.bn_fc1 = nn.BatchNorm1d(512, momentum=0.1, affine=True, track_running_stats = True)
        self.dropout_fc1 = nn.Dropout(p = 0.5)

        self.fc2 = nn.Linear(512,64)
        self.bn_fc2 = nn.BatchNorm1d(64, momentum=0.1, affine=True, track_running_stats = True)
        self.dropout_fc2 = nn.Dropout(p = 0.5)
        self.fc3 = nn.Linear(64,num_speakers)

    def statis_pooling(self,x):
        stats = torch.cat((x.mean(dim=2), x.std(dim=2)), dim=1)
        return stats

    def forward(self, x):#eps是噪声部分
        x = self.TDNN1(x)
        x = self.TDNN2(x)
        x = self.TDNN3(x)
        x = self.TDNN4(x)
        x = self.TDNN5(x)

#         if self.training:
#             x = x + torch.randn(x.size()).cuda()*eps
        stats = self.statis_pooling(x)
        x = self.dropout_fc1(self.bn_fc1(F.relu(self.fc1(stats))))
        x = self.dropout_fc2(self.bn_fc2(F.relu(self.fc2(x))))
        x = self.fc3(x)
        return x
    
    def extract_x_vector(self, x):
        x = self.TDNN1(x)
        x = self.TDNN2(x)
        x = self.TDNN3(x)
        x = self.TDNN4(x)
        x = self.TDNN5(x)
        stats = self.statis_pooling(x)
        xvec = self.fc1(stats)
        return xvec



In [14]:
model = TDNN(43)

In [15]:
x = torch.rand(10, 39, 100)

In [16]:
output = model(x,)

In [18]:
output.shape

torch.Size([10, 43])

In [20]:
model.eval()
x_vector = model.extract_x_vector(x)

In [21]:
x_vector.shape

torch.Size([10, 512])

In [8]:

class TDNN(nn.Module):

    def __init__(self, input_dim, num_speakers, p_dropout):
        super(TDNN, self).__init__()
        self.tdnn1 = nn.Conv1d(in_channels = input_dim, out_channels = 512, kernel_size = 5, padding = 2 ,dilation = 1 )
        self.bn_tdnn1 = nn.BatchNorm1d(512, momentum=0.1, affine=True)
        self.dropout_tdnn1 = nn.Dropout(p=p_dropout)

        self.tdnn2 = nn.Conv1d(in_channels=512, out_channels=512, kernel_size=3, padding = 2,dilation=2)
        self.bn_tdnn2 = nn.BatchNorm1d(512, momentum=0.1, affine=True)
        self.dropout_tdnn2 = nn.Dropout(p=p_dropout)

        self.tdnn3 = nn.Conv1d(in_channels=512, out_channels=512,kernel_size=3, padding = 3,dilation=3)
        self.bn_tdnn3 = nn.BatchNorm1d(512, momentum=0.1, affine=True)
        self.dropout_tdnn3 = nn.Dropout(p=p_dropout)
        
        self.tdnn4 = nn.Conv1d(in_channels=512, out_channels=512,kernel_size=1)
        self.bn_tdnn4 = nn.BatchNorm1d(512, momentum=0.1, affine=True)
        self.dropout_tdnn4 = nn.Dropout(p=p_dropout)

        self.tdnn5 = nn.Conv1d(in_channels=512, out_channels=1536,kernel_size=1)
        self.bn_tdnn5 = nn.BatchNorm1d(1536, momentum=0.1, affine=True)
        self.dropout_tdnn5 = nn.Dropout(p=p_dropout)

        self.fc1 = nn.Linear(2*1536,512)
        self.bn_fc1 = nn.BatchNorm1d(512, momentum=0.1, affine=True)
        self.dropout_fc1 = nn.Dropout(p=p_dropout)

        self.fc2 = nn.Linear(512,128)
        self.bn_fc2 = nn.BatchNorm1d(128, momentum=0.1, affine=True)
        self.dropout_fc2 = nn.Dropout(p=p_dropout)

        self.fc3 = nn.Linear(128,num_speakers)

    def forward(self, x):

        x = self.dropout_tdnn1(self.bn_tdnn1(F.relu(self.tdnn1(x))))
        print(x.shape)
        x = self.dropout_tdnn2(self.bn_tdnn2(F.relu(self.tdnn2(x))))
        print(x.shape)
        x = self.dropout_tdnn3(self.bn_tdnn3(F.relu(self.tdnn3(x))))
        print(x.shape)
        x = self.dropout_tdnn4(self.bn_tdnn4(F.relu(self.tdnn4(x))))
        print(x.shape)
        x = self.dropout_tdnn5(self.bn_tdnn5(F.relu(self.tdnn5(x))))
        print(x.shape)

        stats = torch.cat((x.mean(dim=2), x.std(dim=2)), dim=1)
        x = self.dropout_fc1(self.bn_fc1(F.relu(self.fc1(stats))))
        x = self.dropout_fc2(self.bn_fc2(F.relu(self.fc2(x))))
        x = self.fc3(x)
        return x
    
    def extract_x_vector(self, x):
        x = self.dropout_tdnn1(self.bn_tdnn1(F.relu(self.tdnn1(x))))
        x = self.dropout_tdnn2(self.bn_tdnn2(F.relu(self.tdnn2(x))))
        x = self.dropout_tdnn3(self.bn_tdnn3(F.relu(self.tdnn3(x))))
        x = self.dropout_tdnn4(self.bn_tdnn4(F.relu(self.tdnn4(x))))
        x = self.dropout_tdnn5(self.bn_tdnn5(F.relu(self.tdnn5(x))))

        stats = torch.cat((x.mean(dim=2), x.std(dim=2)), dim=1)
        xv = self.bn_fc1(F.relu(self.fc1(stats)))
        return xv



In [9]:
model = TDNN(39, 43, 0.25)

In [10]:
input_feature = torch.rand(10,39,135)
model(input_feature)

torch.Size([10, 512, 135])
torch.Size([10, 512, 135])
torch.Size([10, 512, 135])
torch.Size([10, 512, 135])
torch.Size([10, 1536, 135])


tensor([[-5.0818e-01, -5.6307e-01, -4.1031e-01,  7.1857e-01,  4.1813e-01,
          5.0056e-01, -2.6477e-01, -8.1584e-02, -2.6281e-01,  1.0420e+00,
          3.2823e-01, -9.1143e-01, -3.8315e-01,  7.1892e-01,  5.0216e-01,
         -4.0544e-02, -9.0196e-01,  1.8183e-01, -1.4102e-01, -3.6217e-01,
         -5.1227e-01,  8.0957e-01, -3.6199e-01,  3.7434e-01, -5.4407e-01,
          6.3408e-01,  9.8175e-01, -7.1631e-01, -4.2326e-01, -7.6716e-01,
          1.2074e+00,  1.7428e-01,  4.4338e-01, -1.0573e+00,  1.1206e+00,
          2.0939e-01,  7.2251e-01, -2.7699e-01, -5.8753e-01,  6.4379e-02,
          2.2755e-01,  5.6865e-01, -1.2766e-01],
        [-1.2177e+00,  1.6044e-02, -8.7840e-01, -3.3230e-01, -1.0604e+00,
         -1.0430e+00, -5.1973e-02,  8.6193e-01,  7.8067e-01,  7.8111e-01,
          2.0963e-01, -8.0671e-01,  3.8851e-01,  1.8397e-03, -3.1295e-01,
         -2.8312e-01, -7.8557e-01, -1.6477e+00, -7.9928e-01,  5.1726e-01,
         -2.8910e-01, -1.3274e+00,  2.1086e+00, -1.0051e+00, -1

In [12]:
x = [1,2]
max(x)

2